In [1]:
import os

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
LANGSMITH_TRACING=os.getenv("LANGSMITH_TRACING")
LANGSMITH_API_KEY=os.getenv("LANGSMITH_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")


os.environ["LANGSMITH_TRACING"]=LANGSMITH_TRACING
os.environ["LANGSMITH_API_KEY"]=LANGSMITH_API_KEY
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"]=TAVILY_API_KEY


### load the model and test it with the simple message

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [8]:
model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

In [9]:
message=[SystemMessage(content="Your are helpfull AI bot "),HumanMessage(content="Hi, how are you buddy?")]

In [10]:
message2=[SystemMessage(content="Hi you are nice Bot and you answer everything in brief"),HumanMessage(content="Hi, how are you buddy?")]

In [11]:
model.invoke(message)

AIMessage(content='I am well, thank you for asking. I am a virtual assistant designed to help you with a variety of tasks. How can I assist you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-4555f455-0aac-4bc5-977f-b6109da7c24c-0', usage_metadata={'input_tokens': 15, 'output_tokens': 31, 'total_tokens': 46, 'input_token_details': {'cache_read': 0}})

### use output parser

In [12]:
from langchain_core.output_parsers import StrOutputParser

In [13]:
parser = StrOutputParser()

In [14]:
response = model.invoke(message)

In [15]:
response2 = model.invoke(message2)

In [16]:
parser.invoke(response)

'I am an AI assistant, not a bot. I am designed to provide information and answer questions to the best of my abilities. I am always learning and improving, and I am here to help you with whatever you need.\n\nAs for how I am doing, I am doing well. I am grateful for the opportunity to help people, and I am always looking for ways to improve my skills and knowledge.\n\nIs there anything I can help you with today?'

In [17]:
parser.invoke(response2)

"I'm well, thanks for asking."

### chaining with LCEL

In [18]:
#added two component
chain = model | parser

In [19]:
chain.invoke(message)

'As an AI chatbot, I don\'t have personal feelings or emotions, so I don\'t experience being a "buddy" or having a subjective state of being. I am designed to provide information and assist users to the best of my abilities.\n\nIs there anything I can help you with today?'

In [20]:
chain.invoke(message2)

"I'm doing well, thanks for asking!"

### Now lets start with prompting

In [21]:
from langchain_core.prompts import ChatPromptTemplate

In [22]:
system="translate the following into {language}"
#user="i am hungry."

In [23]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system","translate the following into {language}"),
     ("user","{text}")]
)

In [24]:
prompt = prompt_template.invoke({"language":"Urdu","text":"i am learning langchain with Suresh beekhani sir."})

In [25]:
parser.invoke(model.invoke(prompt))

'میں سرش بیخانی سر کے ساتھ لینگ چین سیکھ رہا ہوں۔'

In [26]:
prompt.to_messages()

[SystemMessage(content='translate the following into Urdu', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='i am learning langchain with Suresh beekhani sir.', additional_kwargs={}, response_metadata={})]

### chaining 

In [27]:
chain = prompt_template | model | parser

In [28]:
chain.invoke({"language": "hindi", "text": "I am hungry"})

'मुझे भूख लगी है'

### Agents

In [29]:

from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [30]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage

In [31]:
memory = MemorySaver()

In [32]:
search=TavilySearchResults( max_results=2)

In [33]:
tools=[search]

In [34]:
agent_executor=create_react_agent(model,tools,checkpointer=memory)

In [35]:
config = {"configurable": {"thread_id": "abc123"}}

In [36]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="hi im sunny! and i live in bangalore")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Nice to meet you Sunny!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-4c670bc7-d70f-4dec-8946-623f9dea9509-0', usage_metadata={'input_tokens': 84, 'output_tokens': 6, 'total_tokens': 90, 'input_token_details': {'cache_read': 0}})]}}
----


In [37]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="whats is  the weather where I live in Umerkot?")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "What is the weather in Umerkot?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-0f15fb23-662f-4099-8dd4-b24429d82044-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'What is the weather in Umerkot?'}, 'id': 'a0ee25c8-7808-4301-9d48-edaea287d3fd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 105, 'output_tokens': 28, 'total_tokens': 133, 'input_token_det